<a href="https://colab.research.google.com/github/RonaldTuna/TensorIoTAssignment/blob/main/TensorIoT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEPS:**

Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

Apache spark tools locally and necessary tools

Download a review file with a million reviews

Using Jupyter notebook create a program to read the million reviews and get the following

transform date to MM-DD-YYYY format

Save the data into a table (postgres/sql server)
Save the output as a Parquet file


Upload code to Github  and complete Readme.md which anyone can understand

Send Github link to HR



Skills Learning / Tools Used:

    postgres
    workbench/J
    Jupyter notebook
    Apache spark
    Data frame transforms
    









In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9965bb57f0ed4e7a19ef8e1a1fd6a9c659f067c014ea7c42f111ca9b807b56a4
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [12]:
import pyspark
from pyspark.sql import SparkSession

I downloaded the reviews for Android Applications, which contains 2,638,173 reviews.

Load reviews into pyspark dataframe... or ADD?

In [14]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
df = spark.read.json('reviews_Apps_for_Android.json.gz')

Read million reviews with Python and transform date to MM DD YYYY

In [19]:
df.printschema()

AttributeError: ignored

Save data into SQL table/server

Save file as parquet file.